<a href="https://colab.research.google.com/github/routsourav1729/CUB_200_2011/blob/main/gnr_best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/routsourav1729/CUB_200_2011.git


fatal: destination path 'CUB_200_2011' already exists and is not an empty directory.


In [ ]:
!pip install efficientnet-pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=0f3659657a45bf87c30d47f874463d2101926efda7430233de20969161a2dee3
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch


In [ ]:
!pip install "torchvision>0.16.0"

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torch import optim
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:


def get_data_loaders(traindata_dir, testdata_dir, batch_size=32, num_workers=0, valid_size=0.1):
    image_transforms = {
        # Train uses data augmentation
        'train': transforms.Compose([
            transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
            transforms.RandomRotation(degrees=15),
            transforms.ColorJitter(),
            transforms.RandomHorizontalFlip(),
            transforms.CenterCrop(size=224),  # ImageNet standards
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet standards
        ]),
        # Test does not use augmentation
        'test': transforms.Compose([
            transforms.Resize(size=256),
            transforms.CenterCrop(size=224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }

    # Define datasets
    dataset = {
        'train': datasets.ImageFolder(root=traindata_dir, transform=image_transforms['train']),
        'test': datasets.ImageFolder(root=testdata_dir, transform=image_transforms['test'])
    }

    # Splitting dataset into training and validation
    num_test = len(dataset['train'])
    indices = list(range(num_test))
    split = int(np.floor(valid_size * num_test))

    np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
    valid_sampler = torch.utils.data.SubsetRandomSampler(valid_idx)

    # Define dataloaders
    dataloaders = {
        'train': DataLoader(dataset['train'], batch_size=batch_size,sampler=train_sampler, num_workers=num_workers),
        'test': DataLoader(dataset['test'], batch_size=batch_size,shuffle=True, num_workers=num_workers),
        'val': DataLoader(dataset['train'], batch_size=batch_size,sampler=valid_sampler, num_workers=num_workers)
    }

    return dataloaders['train'], dataloaders['val'], dataloaders['test']


In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# Initialize the EfficientNet-B2 model with pre-trained weights
model = models.efficientnet_b1(weights='IMAGENET1K_V1')

# Determine the number of layers in the model
num_layers = len(list(model.children()))

# Specify the number of layers to freeze (initial convolutional layers)
freeze_layers = 0

# Freeze the initial convolutional layers
for i, (name, param) in enumerate(model.named_parameters()):
    if 'conv_stem' in name or i < freeze_layers:
        param.requires_grad = False

# Unfreeze the deeper layers
for i, (name, param) in enumerate(model.named_parameters()):
    if 'conv_stem' not in name and i >= freeze_layers:
        param.requires_grad = True

# Check which parameters are trainable
for name, param in model.named_parameters():
    print(name, param.requires_grad)


In [ ]:
print(model)


In [ ]:
# Add on classifier
model.classifier[1] = nn.Linear(1280, 200)

total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

6,769,384 total parameters.
6,769,384 training parameters.


In [ ]:
train_dir = '/content/CUB_200_2011/train'
test_dir = '/content/CUB_200_2011/test'

train_loader, val_loader,test_loader = get_data_loaders(train_dir,test_dir)

In [ ]:


# Initialize model and move to device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.0001,weight_decay=1e-5)

# Training the model
num_epochs = 20
training_loss_list = []
validation_accuracy_list = []
training_accuracy_list = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    average_loss = total_loss / len(train_loader)
    training_loss_list.append(average_loss)
    training_accuracy = 100 * correct_train / total_train
    training_accuracy_list.append(training_accuracy)

    model.eval()
    with torch.no_grad():
        all_preds = []
        all_labels = []
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100*accuracy_score(all_labels, all_preds)
    validation_accuracy_list.append(accuracy)

    print(f"Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {average_loss:.4f}, Train Accuracy: {training_accuracy:.2f}%, validation Accuracy: {accuracy:.4f}%")


Epoch 1/20, Average Training Loss: 5.0447, Train Accuracy: 5.99%, validation Accuracy: 18.6978%
Epoch 2/20, Average Training Loss: 3.7722, Train Accuracy: 32.34%, validation Accuracy: 37.5626%
Epoch 3/20, Average Training Loss: 2.5870, Train Accuracy: 52.97%, validation Accuracy: 51.5860%
Epoch 4/20, Average Training Loss: 1.8352, Train Accuracy: 66.25%, validation Accuracy: 58.2638%
Epoch 5/20, Average Training Loss: 1.3632, Train Accuracy: 74.96%, validation Accuracy: 65.4424%
Epoch 6/20, Average Training Loss: 1.0273, Train Accuracy: 80.93%, validation Accuracy: 68.6144%
Epoch 7/20, Average Training Loss: 0.7857, Train Accuracy: 84.99%, validation Accuracy: 71.6194%
Epoch 8/20, Average Training Loss: 0.6310, Train Accuracy: 88.71%, validation Accuracy: 73.4558%
Epoch 9/20, Average Training Loss: 0.5104, Train Accuracy: 90.38%, validation Accuracy: 74.4574%
Epoch 10/20, Average Training Loss: 0.3968, Train Accuracy: 93.59%, validation Accuracy: 74.1235%
Epoch 11/20, Average Training 

In [ ]:
# Testing the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total}%')

Test Accuracy: 80.47980669658267%


In [ ]:
print(f'Training Accuracy List: {training_accuracy_list}')
print(f'Validation Accuracy List: {validation_accuracy_list}')
print(f'Training Loss List: {training_loss_list}')


Training Accuracy List: [15.98264931598265, 44.07741074407741, 61.378044711378045, 71.25458792125458, 78.41174507841174, 83.667000333667, 86.48648648648648, 89.72305638972306, 92.75942609275943, 93.7771104437771, 95.42876209542877, 96.3963963963964, 96.91358024691358, 97.83116449783117, 98.54854854854855, 98.84884884884885, 98.98231564898232, 98.81548214881548, 99.18251584918252, 99.33266599933266]
Validation Accuracy List: [37.99654576856649, 57.167530224525045, 67.18480138169257, 72.19343696027633, 76.16580310880829, 75.82037996545769, 78.23834196891191, 77.54749568221071, 78.23834196891191, 79.96545768566494, 80.31088082901555, 81.00172711571675, 80.31088082901555, 79.96545768566494, 80.82901554404145, 79.96545768566494, 81.51986183074266, 80.13816925734024, 80.82901554404145, 80.31088082901555]
Training Loss List: [4.558079941475645, 3.085178770917527, 2.110614814656846, 1.5181881626869769, 1.1324887133025108, 0.8556809365115268, 0.6810854961897465, 0.5386697537404426, 0.4118055029

In [ ]:
torch.save( model.state_dict(), "/content/CUB_200_2011/Model_State_ENB1.pth")

In [ ]:
from efficientnet_pytorch import EfficientNet
import torch.nn as nn

# Initialize EfficientNet B1
model_b1 = models.efficientnet_b1(weights='IMAGENET1K_V1')


# Assume the feature extraction part of B1 matches B0 and can accept the weights
# Modify the classifier layer as per your requirement
model_b1.classifier[1] = nn.Linear(1280, 200)

total_params = sum(p.numel() for p in model_b1.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in model_b1.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

Loaded pretrained weights for efficientnet-b1
6,769,384 total parameters.
6,769,384 training parameters.


In [ ]:

model_weights_path = '/content/CUB_200_2011/Model_State_ENB1.pth'

# Load the state_dict of the B0 model
b0_state_dict = torch.load(model_weights_path)

# Load with strict=False to ignore incompatible keys
model_b1.load_state_dict(b0_state_dict, strict=False)


In [ ]:
# Assuming test_loader is already defined
model_b1.eval()  # Set the model to evaluation mode
correct = 0
total = 0
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_b1.to(device)

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model_b1(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy of EfficientNet B1: {accuracy}%')

Test Accuracy of EfficientNet B1: 0.25888850535036245%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt

# Assuming CUB-200-2011 has 200 classes
num_classes = 200
batch_size = 32
num_epochs = 20
learning_rate = 0.0001

# Image transformations
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=299, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=30),
        transforms.RandomRotation(degrees=45),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=299),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),

    'test':
    transforms.Compose([
        transforms.Resize(size=299),
        transforms.CenterCrop(size=299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


# Datasets from folders
data = {
    'train':
    datasets.ImageFolder(root='/content/CUB_200_2011/train', transform=image_transforms['train']),
    'test':
    datasets.ImageFolder(root='/content/CUB_200_2011/test', transform=image_transforms['test']),
}

# Dataloader iterators, make sure to shuffle

train_loader = DataLoader(data['train'], batch_size=batch_size, shuffle=True)
test_loader = DataLoader(data['test'], batch_size=batch_size, shuffle=False)




In [ ]:
model = models.efficientnet_b1(weights='IMAGENET1K_V1')
model.classifier[1] = nn.Linear(1280, 200)


total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

6,769,384 total parameters.
6,769,384 training parameters.


In [ ]:


# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5)

# Training the model
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_acc:.2f}%')


print('Finished Training')

# Testing the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total}%')


Epoch [1/20], Loss: 4.5666, Train Accuracy: 14.06%
Epoch [2/20], Loss: 3.1952, Train Accuracy: 35.39%
Epoch [3/20], Loss: 2.2661, Train Accuracy: 55.69%
Epoch [4/20], Loss: 1.6936, Train Accuracy: 66.97%
Epoch [5/20], Loss: 1.2945, Train Accuracy: 74.42%
Epoch [6/20], Loss: 1.0241, Train Accuracy: 79.13%
Epoch [7/20], Loss: 0.8330, Train Accuracy: 82.23%
Epoch [8/20], Loss: 0.6746, Train Accuracy: 85.57%
Epoch [9/20], Loss: 0.5782, Train Accuracy: 87.35%
Epoch [10/20], Loss: 0.4823, Train Accuracy: 90.41%
Epoch [11/20], Loss: 0.4148, Train Accuracy: 91.27%
Epoch [12/20], Loss: 0.3481, Train Accuracy: 92.79%
Epoch [13/20], Loss: 0.3007, Train Accuracy: 93.74%
Epoch [14/20], Loss: 0.2545, Train Accuracy: 95.15%
Epoch [15/20], Loss: 0.2132, Train Accuracy: 96.26%
Epoch [16/20], Loss: 0.1916, Train Accuracy: 96.38%
Epoch [17/20], Loss: 0.1753, Train Accuracy: 96.98%
Epoch [18/20], Loss: 0.1429, Train Accuracy: 97.33%
Epoch [19/20], Loss: 0.1243, Train Accuracy: 97.83%
Epoch [20/20], Loss: 

In [ ]:
torch.save( model.state_dict(), "/content/CUB_200_2011/Model_State_ENB2.pth")

In [ ]:
# Initialize EfficientNet B1
model_b1 = models.efficientnet_b1(weights='IMAGENET1K_V1')


# Assume the feature extraction part of B1 matches B0 and can accept the weights
# Modify the classifier layer as per your requirement
model_b1.classifier[1] = nn.Linear(1280, 200)

total_params = sum(p.numel() for p in model_b1.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in model_b1.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

6,769,384 total parameters.
6,769,384 training parameters.


In [ ]:

model_weights_path = '/content/CUB_200_2011/Model_State_ENB2.pth'

# Load the state_dict of the B0 model
b0_state_dict = torch.load(model_weights_path)

# Load with strict=False to ignore incompatible keys
model_b1.load_state_dict(b0_state_dict, strict=False)


<All keys matched successfully>

In [ ]:
# Assuming test_loader is already defined
model_b1.eval()  # Set the model to evaluation mode
correct = 0
total = 0
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_b1.to(device)

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model_b1(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy of EfficientNet B1: {accuracy}%')

Test Accuracy of EfficientNet B1: 81.32550914739386%
